# 获取各视频评论数据

已有视频链接信息，获取详细评论信息（不包括对评论的评论），每个视频文件评论单独成文件，清洗后的结果保存在 `../data/reply/reply_by_bvid_raw/` 中。

In [1]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import time
import datetime
import pandas as pd
import numpy as np
import os

In [2]:
def get_comment_raw(oid, maxpage):
    rs = []
    time.sleep(np.random.uniform(low=0.1, high=0.5, size=1).item())
    for i in range(1, maxpage):
        time.sleep(np.random.uniform(low=0.05, high=0.4, size=1).item())
        url = "https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next={}&type=1&oid={}&sort=2".format(i, oid)
        payload={}
        headers = {}
        response = requests.request("GET", url, headers=headers, data=payload)
        r = response.text
        rs.append(r)
    return rs

In [3]:
def parse_comment(comment_onepage):
    replies = []
    for i, r in enumerate(comment_onepage):
        comm_text = json.loads(r)
        try:
            replies.extend(comm_text['data']["replies"])
        except:
            continue
    return replies

In [4]:
df = pd.read_csv("../data/video/video_final.csv")
df_use = df.iloc[:,:].reset_index()
maxpages = (df_use['reply']//20 + 2).to_list() # 评论最大页数

In [ ]:
vid_done = [t[6:18] for t in os.listdir('../data/reply/reply_by_bvid_raw')]
len(vid_done), len(df['v_id']), len(set(df['v_id']).difference(set(vid_done)))
# 代码中途出错后，只再获取没有拿到评论的视频，对应 id 保存在 vid_left 中
# 若视频评论区设置不可见，则无法获取信息

In [8]:
vid_left = list(set(df['v_id']).difference(set(vid_done)))
vid_left

['BV1LW411L7h7', 'BV1WW411g7qW']

In [9]:
df_use = df[df['v_id'].isin(vid_left)].reset_index()
maxpages = (df_use['reply']//20 + 2).to_list()

In [10]:
f = open("../logs/comment_log.txt", "a")
for i, vid in enumerate(df_use['v_id']):
    print(np.where(df['v_id'] == vid)[0][0],df_use['aid'][i], vid, maxpages[i])
    cr = get_comment_raw(df_use['aid'][i], maxpage = maxpages[i])
    cm = parse_comment(cr)
    vdf = pd.DataFrame(cm)
    if vdf.shape[0] < 2:
        print(vid)
        continue
    vdf.to_csv("../data/reply/reply_{}.csv".format(vid), index=False, header=True, encoding="UTF-8")
    f.write("\n{}\t num:{}\t vid:{}\t len:{}".format(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                        i, vid, vdf.shape[0]))
    print("../data/reply_{}.csv".format(vid))
f.write("\n====================================================================")
f.close()

174 22041882 BV1LW411L7h7 64
BV1LW411L7h7
285 21097888 BV1WW411g7qW 2
BV1WW411g7qW
